In [1]:
import spacy
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
)
import pandas as pd
import numpy as np
import pandas as pd
import re
import spacy
from spacy.tokens import DocBin
from spacy.cli.train import train

In [2]:
nlp = spacy.blank("pt")
df_test = pd.read_csv('../../../dataset/v1/test.csv')

labels = df_test.columns[3:].to_list()
y_test = df_test[labels].to_numpy()

In [3]:
# ignore score since we use dummy data for dev
train('./bow_config.cfg', output_path='./output_bow')

ℹ Saving to output directory: output_bow
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  CATS_MACRO_P  CATS_MACRO_R  CATS_MACRO_F  SCORE 
---  ------  -------------  ----------  ------------  ------------  ------------  ------
  0       0           0.25        0.00         25.00         25.00         25.00    0.25
  0     200          44.49        0.00         25.00         25.00         25.00    0.25
  0     400          43.18        0.00          0.00          0.00          0.00    0.00
  0     600          39.73        0.00          0.00          0.00          0.00    0.00
  0     800          37.18        0.00         25.00         25.00         25.00    0.25
  1    1000          17.69        0.00         12.50         12.50         12.50

In [4]:
nlp = spacy.load('./output_bow/model-last')
nlp.pipe_names

['textcat_multilabel']

In [5]:
nlp('I like this game so much').cats

{'label_recommended': 0.48984286189079285,
 'label_story': 0.42084258794784546,
 'label_gameplay': 0.4618592858314514,
 'label_visual': 0.46005651354789734,
 'label_audio': 0.4021838307380676,
 'label_technical': 0.4266018271446228,
 'label_price': 0.49449148774147034,
 'label_suggestion': 0.39189016819000244}

In [6]:
y_pred = np.zeros(shape=y_test.shape)
for i, doc in enumerate(nlp.pipe(df_test['cleaned_review'].to_list())):
    y_pred[i, :] = list(doc.cats.values())
mask_pos = y_pred > 0.5
y_pred[mask_pos] = 1.0
y_pred[~mask_pos] = 0.0
y_pred

array([[1., 0., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

In [7]:
def evaluate(y_test, y_pred, labels):
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Overall accuracy: {accuracy}')
    for idx, label in enumerate(labels):
        label_accuracy = accuracy_score(y_test[:, idx], y_pred[:, idx])
        print(f'Accuracy {label}: {label_accuracy}')

    f1 = f1_score(y_test, y_pred, average='macro')
    print(f'F1 macro: {f1}')
    print(
        classification_report(y_test, y_pred, target_names=labels, digits=4, zero_division=0)
    )

evaluate(y_test, y_pred, labels)

Overall accuracy: 0.17
Accuracy label_recommended: 0.78
Accuracy label_story: 0.77
Accuracy label_gameplay: 0.81
Accuracy label_visual: 0.715
Accuracy label_audio: 0.77
Accuracy label_technical: 0.8
Accuracy label_price: 0.775
Accuracy label_suggestion: 0.89
F1 macro: 0.5493574741399815
                   precision    recall  f1-score   support

label_recommended     0.7989    0.9392    0.8634       148
      label_story     0.7471    0.7303    0.7386        89
   label_gameplay     0.8295    0.9481    0.8848       154
     label_visual     0.6471    0.7586    0.6984        87
      label_audio     0.6923    0.1765    0.2812        51
  label_technical     0.6977    0.5263    0.6000        57
      label_price     0.5500    0.2340    0.3284        47
 label_suggestion     0.0000    0.0000    0.0000        21

        micro avg     0.7565    0.7125    0.7339       654
        macro avg     0.6203    0.5391    0.5494       654
     weighted avg     0.7182    0.7125    0.6950       654
  